# Procedures

Record recycling transaction

In [2]:
CREATE PROCEDURE recycling.usp_RecordRecyclingTransaction
    @BusinessID INT,
    @UserID INT,
    @TransactionType NVARCHAR(20),
    @PaymentAmount DECIMAL(10,2),
    @PaymentCurrency NVARCHAR(3),
    @Notes NVARCHAR(500),
    @Materials NVARCHAR(MAX) -- JSON array of {materialCategoryID, quantity, unit, pricePerUnit}
AS
BEGIN
    SET NOCOUNT ON;
    
    BEGIN TRANSACTION;
    
    BEGIN TRY
        -- Insert main transaction
        DECLARE @TransactionID INT;
        
        INSERT INTO recycling.RecyclingTransactions (
            BusinessID, UserID, TransactionType, 
            PaymentAmount, PaymentCurrency, PaymentStatus, Notes
        )
        VALUES (
            @BusinessID, @UserID, @TransactionType,
            @PaymentAmount, @PaymentCurrency, 'COMPLETED', @Notes
        );
        
        SET @TransactionID = SCOPE_IDENTITY();
        
        -- Insert transaction materials
        INSERT INTO recycling.TransactionMaterials (
            TransactionID, MaterialCategoryID, 
            Quantity, Unit, PricePerUnit, TotalPrice
        )
        SELECT 
            @TransactionID,
            JSON_VALUE(m.value, '$.materialCategoryID'),
            JSON_VALUE(m.value, '$.quantity'),
            JSON_VALUE(m.value, '$.unit'),
            JSON_VALUE(m.value, '$.pricePerUnit'),
            CAST(JSON_VALUE(m.value, '$.quantity') AS DECIMAL(10,2)) * 
            CAST(JSON_VALUE(m.value, '$.pricePerUnit') AS DECIMAL(10,2))
        FROM OPENJSON(@Materials) m;
        
        COMMIT TRANSACTION;
        
        SELECT @TransactionID AS TransactionID;
    END TRY
    BEGIN CATCH
        ROLLBACK TRANSACTION;
        THROW;
    END CATCH
END;
GO 

: Msg 2714, Level 16, State 3, Procedure usp_RecordRecyclingTransaction, Line 1
There is already an object named 'usp_RecordRecyclingTransaction' in the database.

Total execution time: 00:01:05.271

Upsert business

In [ ]:
CREATE PROCEDURE recycling.usp_UpsertBusiness
    @Name NVARCHAR(255),
    @Address NVARCHAR(500),
    @Latitude DECIMAL(10, 8),
    @Longitude DECIMAL(11, 8),
    @Phone NVARCHAR(50),
    @Website NVARCHAR(500),
    @Rating DECIMAL(3, 2),
    @PlaceID NVARCHAR(255),
    @Materials NVARCHAR(MAX)  -- JSON array of material categories
AS
BEGIN
    SET NOCOUNT ON;
    
    DECLARE @BusinessID INT;
    
    -- Insert or update business
    MERGE recycling.Businesses AS target
    USING (SELECT @PlaceID AS PlaceID) AS source
    ON target.PlaceID = source.PlaceID
    WHEN MATCHED THEN
        UPDATE SET 
            Name = @Name,
            FormattedAddress = @Address,
            Latitude = @Latitude,
            Longitude = @Longitude,
            PhoneNumber = @Phone,
            Website = @Website,
            Rating = @Rating,
            LastUpdated = SYSUTCDATETIME()
    WHEN NOT MATCHED THEN
        INSERT (Name, FormattedAddress, Latitude, Longitude, PhoneNumber, Website, Rating, PlaceID)
        VALUES (@Name, @Address, @Latitude, @Longitude, @Phone, @Website, @Rating, @PlaceID)
    OUTPUT INSERTED.BusinessID INTO @BusinessID;

    -- Clear and update materials if specified
    IF @Materials IS NOT NULL
    BEGIN
        DELETE FROM recycling.BusinessMaterials WHERE BusinessID = @BusinessID;

        -- Insert materials based on JSON array
        INSERT INTO recycling.BusinessMaterials (BusinessID, CategoryID)
        SELECT @BusinessID, CategoryID
        FROM recycling.MaterialCategories
        WHERE CategoryName IN (SELECT value FROM OPENJSON(@Materials));
    END
END;
GO 

Verify business

In [ ]:
CREATE PROCEDURE recycling.usp_VerifyBusiness
    @BusinessID INT,
    @Email NVARCHAR(255),
    @BusinessName NVARCHAR(255),
    @Address NVARCHAR(500),
    @Phone NVARCHAR(50),
    @Website NVARCHAR(500),
    @Materials NVARCHAR(MAX)  -- JSON array of material categories
AS
BEGIN
    SET NOCOUNT ON;
    
    BEGIN TRANSACTION;
    
    BEGIN TRY
        -- Update business information
        UPDATE recycling.Businesses
        SET Name = @BusinessName,
            FormattedAddress = @Address,
            PhoneNumber = @Phone,
            Website = @Website,
            LastUpdated = SYSUTCDATETIME()
        WHERE BusinessID = @BusinessID;

        -- Create or update business account
        MERGE recycling.BusinessAccounts AS target
        USING (SELECT @BusinessID AS BusinessID, @Email AS Email) AS source
        ON target.BusinessID = source.BusinessID
        WHEN MATCHED THEN
            UPDATE SET 
                Email = @Email,
                ModifiedDate = SYSUTCDATETIME()
        WHEN NOT MATCHED THEN
            INSERT (BusinessID, Email)
            VALUES (@BusinessID, @Email);

        -- Update materials handled
        IF @Materials IS NOT NULL
        BEGIN
            DELETE FROM recycling.BusinessMaterials 
            WHERE BusinessID = @BusinessID;

            INSERT INTO recycling.BusinessMaterials (BusinessID, CategoryID, IsVerified)
            SELECT @BusinessID, CategoryID, 1
            FROM recycling.MaterialCategories
            WHERE CategoryName IN (SELECT value FROM OPENJSON(@Materials));
        END

        COMMIT TRANSACTION;
    END TRY
    BEGIN CATCH
        ROLLBACK TRANSACTION;
        THROW;
    END CATCH
END;
GO 